# Train VoxNet (http://dimatura.net/research/voxnet/).

This notebook shows you how to use the ETLDataGenerator in order to train VoxNet. 

In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import os
import random

# Get the dataset path.

This snippet shows you how to get the lates preprocessed path.

In [ ]:
from cgmcore.etldatagenerator import get_dataset_path

dataset_path = get_dataset_path("../../data/etl")
print("Using dataset path", dataset_path)

# Hyperparameters.

In [ ]:
steps_per_epoch = 10
validation_steps = 10
epochs = 4
batch_size = 1
random_seed = 667

# Create data-generator.

The method create_datagenerator_from_parameters is a convencience method. It allows you to instantiate a generator from a specification-dictionary.

In [ ]:
from cgmcore.etldatagenerator import create_datagenerator_from_parameters

dataset_parameters_voxelgrids = {}
dataset_parameters_voxelgrids["input_type"] = "voxelgrid"
dataset_parameters_voxelgrids["output_targets"] = ["height"]
dataset_parameters_voxelgrids["random_seed"] = random_seed
dataset_parameters_voxelgrids["voxelgrid_target_shape"] = (32, 32, 32)
dataset_parameters_voxelgrids["voxel_size_meters"] = 0.1
dataset_parameters_voxelgrids["voxelgrid_random_rotation"] = False
dataset_parameters_voxelgrids["sequence_length"] = 0
datagenerator_instance_voxelgrids = create_datagenerator_from_parameters(dataset_path, dataset_parameters_voxelgrids)

# Getting the QR-Codes and do a train-validate-split.

The data-generator is perfectly capable of retrieving all QR-codes from the dataset. This snippet shows how to do so and how to split the QR-codes into two sets: Train and validate.

In [ ]:
# Get the QR-codes.
qrcodes_to_use = datagenerator_instance_voxelgrids.qrcodes[0:30]

# Do the split.
random.seed(random_seed)
qrcodes_shuffle = qrcodes_to_use[:]
random.shuffle(qrcodes_shuffle)
split_index = int(0.8 * len(qrcodes_shuffle))
qrcodes_train = sorted(qrcodes_shuffle[:split_index])
qrcodes_validate = sorted(qrcodes_shuffle[split_index:])
del qrcodes_shuffle
print("QR-codes for training:\n", "\t".join(qrcodes_train))
print("QR-codes for validation:\n", "\t".join(qrcodes_validate))

# Creating python generators for training and validation.

Now both QR-codes lists can be used for creating the actual generators. One for training and one for validation.

In [ ]:
# Create python generators.
generator_voxelgrids_train = datagenerator_instance_voxelgrids.generate(size=batch_size, qrcodes_to_use=qrcodes_train)
generator_voxelgrids_validate = datagenerator_instance_voxelgrids.generate(size=batch_size, qrcodes_to_use=qrcodes_validate)

# Using the generator to create data manually.

Of course you can use the generator to create data manually anytime.

In [ ]:
train_x, train_y = next(generator_voxelgrids_train)
print("Input-shape:", train_x.shape)
print("Output-shape:", train_y.shape)

# Training-details.

Training-details are a dictionary that gets stored in a file after training. It is supposed to contain information that is valuable. For example data that is relevant for training including the hyper-parameters. Intended to be used when comparing different models.

In [ ]:
training_details = {
    "dataset_path" : dataset_path,
    "qrcodes_train" : qrcodes_train,
    "qrcodes_validate" : qrcodes_validate,
    "steps_per_epoch" : steps_per_epoch,
    "validation_steps" : validation_steps,
    "epochs" : epochs,
    "batch_size" : batch_size,
    "random_seed" : random_seed,
}

# Training VoxNet.

The module modelutils contains methods for creating Neural Nets. The following code shows how to instantiate and train VoxNet.

In [ ]:
from cgmcore import modelutils

input_shape = (32, 32, 32)
output_size = 1
model_voxnet = modelutils.create_voxnet_model_homepage(input_shape, output_size)
model_voxnet.summary()

# Compile the model.
model_voxnet.compile(
    optimizer="rmsprop",
    loss="mse",
    metrics=["mae"]
    )

# Train the model.
history = model_voxnet.fit_generator(
    generator_voxelgrids_train,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=generator_voxelgrids_validate,
    validation_steps=validation_steps
    )

# Saving everything.

This saves the model, its history and the training-details to some output directory. The created artifacts can later be uses in order to compare different models.

In [ ]:
output_path = "."

modelutils.save_model_and_history(output_path, model_voxnet, history, training_details, "voxnet")